In [ ]:
import requests
import pandas as pd
from io import StringIO
response = requests.get('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=20200120&type=ALLBUT0999&_=1520785530355')

In [ ]:
lines = pd.read_csv(StringIO(response.text))

In [ ]:
lines = response.text.split('\n')
lines[1]

In [ ]:
newlines = []
for line in lines:
    if len(line.split('",')) == 17:
            newlines.append(line)

print(len(lines))
print(len(newlines))



In [ ]:
print(newlines)

In [ ]:
s = '\r'.join(newlines)
print(len(s))
print(s)
type(s)

In [ ]:
df = pd.read_csv(StringIO(s)) # 將 s 用StringIO變成檔案，並用 pd.read_csv 來讀取檔案
df

In [ ]:
s = s.replace('=', '')
df = pd.read_csv(StringIO(s)) 
df

In [ ]:
print(type(df['證券代號'][0]))
print(type(df['證券名稱'][0]))
print(type(df['成交筆數'][0]))
print(type(df['開盤價'][0]))  # 是str而非float
print(type(df['漲跌(+/-)'][0]))  # 是str而非float
print(type(df['Unnamed: 16'][0]))  # 是numpy.float64



## 還有一些空值、Unnamed:16

In [ ]:
df = df.astype(str)
df = df.applymap(lambda s: s.replace(',', '')) 
# apply: 讓方程作用於DataFrame中的每一個元素
# lamba: 輸入左邊、輸出右邊
df

In [ ]:
df = df.set_index('證券代號')
df

In [ ]:
df = df.apply(lambda s: pd.to_numeric(s, errors='coerce')) #將資料轉回數字，coerce就是回傳NaN
df

In [ ]:
df.dropna(axis=1, how='all', inplace=True) # 刪掉空值、缺值的column
"""
axis:
axis=0: 刪除包含缺值的row
axis=1: 刪除包含缺值的column
how: 與axis配合使用
how=‘any’ :只要有缺值出現，就刪除該行或列
how=‘all’: 所有的值都缺，才刪除行或列
inplace: 是否在原資料上操作。如果為真，返回None否則返回新的copy，去掉了缺失值
"""
df



In [ ]:
close_open = df['收盤價'] / df['開盤價']
close_open.head()

In [ ]:
df[close_open > 1.07] # 漲幅超過7%的

In [ ]:
df.to_csv('daily_price.csv', encoding='utf_8_sig')

# 讀檔
# 我們指名 index 為 證券代號
df = pd.read_csv('daily_price.csv', index_col=['證券代號'])

print('index為證券代號')
print('     v')
df.head()

In [ ]:
# 將 sql 通道打開
import sqlite3
conn = sqlite3.connect('test.sqlite3')

# 存檔 if_exists='replace' 是說假如sql中已經有 daily_price 這個 dataframe，則取代它
df.to_sql('daily_price', conn, if_exists='replace')

# 讀檔
df = pd.read_sql('select * from daily_price', conn, index_col=['證券代號'])
df